In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import profanity_check

<p>Загружаем данные...</p>

In [2]:
players = pd.read_csv('/data/notebook_files/players.csv')
chat = pd.read_csv('/data/notebook_files/chat.csv')

<h2>Обработка сообщений из чата</h2>

<p>Предварительная обработка данных - номера слотов в players закодированы немного иначе.</p>

In [3]:
players["player_slot"] = players["player_slot"].replace(128, 5)
players["player_slot"] = players["player_slot"].replace(129, 6)
players["player_slot"] = players["player_slot"].replace(130, 7)
players["player_slot"] = players["player_slot"].replace(131, 8)
players["player_slot"] = players["player_slot"].replace(132, 9)

<p>Объединяем данные в одну таблицу и выбираем только нужные нам атрибуты.</p>

In [4]:
merged = chat.merge(players, 'left', on = ["match_id", "player_slot"])

attrib = ["match_id", "account_id", "player_slot", "key"]
merged = merged[attrib]
merged

,match_id,account_id,player_slot,key
0,0,0.0,6,force it
1,0,1.0,1,space created
2,0,1.0,1,hah
3,0,0.0,6,ez 500
4,0,3.0,4,mvp ulti
...,...,...,...,...
1439483,49999,0.0,8,ez
1439484,49999,0.0,1,ya ya so ezx
1439485,49999,0.0,8,ez
1439486,49999,0.0,1,hahaha


<p>В таблице есть NaN значения, например, в account_id.</p>

In [5]:
merged[merged["account_id"].isna()]

,match_id,account_id,player_slot,key
245074,8216,NaN,-9,зарание спасибо
332431,11215,NaN,-9,nazi army
489758,16613,NaN,-9,to ez
652287,22654,NaN,-9,ggwp


<p>Уберем все строки с NaN значениями, в итоге были убраны только 18 строк. Это не отразится на качестве данных.</p>

In [6]:
merged = merged.dropna()
merged

,match_id,account_id,player_slot,key
0,0,0.0,6,force it
1,0,1.0,1,space created
2,0,1.0,1,hah
3,0,0.0,6,ez 500
4,0,3.0,4,mvp ulti
...,...,...,...,...
1439483,49999,0.0,8,ez
1439484,49999,0.0,1,ya ya so ezx
1439485,49999,0.0,8,ez
1439486,49999,0.0,1,hahaha


<p>Подсчитаем, сколько сообщений отправил в общий чат каждый игрок. Особенно интересует количество сообщений от анонимных игроков (account_id = 0).</p>

In [7]:
messages = merged.groupby("account_id")["account_id"].count().sort_values(ascending=False)
messages = messages.rename("message_count")
messages

<p>Посчитаем, в скольки матчах из датасета присутствует игрок. Для анонимных игроков (account_id = 0) считается, сколько раз в датасете встречаются анонимные игроки.</p>

In [8]:
matches_played = players.groupby("account_id")["account_id"].count().sort_values(ascending=False)
matches_played = matches_played.rename("matches_count")
matches_played

<p>Демонстрация работы библиотеки alt-profanity-check, которая определяет наличие нецензурных слов или направленных на разжигание ненависти сообщений.</p>

In [9]:
from profanity_check import predict, predict_prob

predict(["I want to kill you"])

<p>Анализируем записи из общего чата на наличие нецензурных слов или направленных на разжигание ненависти сообщений.</p>

In [10]:
arr = merged["key"].array
data = predict(arr)
profanity = pd.Series(data).rename("profanity")
profanity

<p>Добавляем полученную информацию к списку аккаунтов игроков.</p>

In [11]:
merged = merged.assign(profanity=profanity)
merged

,match_id,account_id,player_slot,key,profanity
0,0,0.0,6,force it,0.0
1,0,1.0,1,space created,0.0
2,0,1.0,1,hah,0.0
3,0,0.0,6,ez 500,0.0
4,0,3.0,4,mvp ulti,0.0
...,...,...,...,...,...
1439483,49999,0.0,8,ez,NaN
1439484,49999,0.0,1,ya ya so ezx,NaN
1439485,49999,0.0,8,ez,NaN
1439486,49999,0.0,1,hahaha,NaN


<p>Посчитаем, сколько плохих сообщений отправил каждый игрок:</p>

In [12]:
fouls = merged.groupby("account_id", as_index=False)["profanity"].sum()
fouls.sort_values(by='profanity', ascending=False)

,account_id,profanity
0,0.0,36622.0
2337,2962.0,162.0
7010,9149.0,83.0
15124,20197.0,79.0
13043,17315.0,59.0
...,...,...
44232,60045.0,0.0
44231,60043.0,0.0
44230,60041.0,0.0
44228,60039.0,0.0


<h2>Анализ покинутых игр</h2>

<p>Посчитаем, в скольких матчах игрок отключался:</p>

In [13]:
disconnected = players[["match_id", "account_id", "leaver_status"]]
disconnected["leaver_status"] = players["leaver_status"].mask(players["leaver_status"].ne(1))
disconnected = disconnected.groupby("account_id", as_index=False)["leaver_status"].count()
disconnected.rename(columns = {'leaver_status':'disconnected'}, inplace = True)
disconnected.sort_values(by='disconnected', ascending=False)

,account_id,disconnected
0,0,2353
3690,3690,5
9038,9038,5
4451,4451,5
25994,25994,4
...,...,...
54014,54014,0
54015,54015,0
54017,54017,0
54018,54018,0


<p>Посчитаем, в скольких матчах игрок отключался на долгое время:</p>

In [14]:
long_disconnected = players[["match_id", "account_id", "leaver_status"]]
long_disconnected["leaver_status"] = players["leaver_status"].mask(players["leaver_status"].ne(2))
long_disconnected = long_disconnected.groupby("account_id", as_index=False)["leaver_status"].count()
long_disconnected.rename(columns = {'leaver_status':'long_disconnected'}, inplace = True)
long_disconnected.sort_values(by='long_disconnected', ascending=False)

,account_id,long_disconnected
0,0,1282
32412,32412,3
5584,5584,2
4817,4817,2
139136,139136,2
...,...,...
53083,53083,0
53084,53084,0
53085,53085,0
53086,53086,0


<p>Посчитаем, в скольких матчах игрок вручную покидал игру:</p>

In [15]:
left = players[["match_id", "account_id", "leaver_status"]]
left["leaver_status"] = players["leaver_status"].mask(players["leaver_status"].ne(3))
left = left.groupby("account_id", as_index=False)["leaver_status"].count()
left.rename(columns = {'leaver_status':'left'}, inplace = True)
left.sort_values(by='left', ascending=False)

,account_id,left
0,0,634
14902,14902,3
12847,12847,3
2543,2543,2
12657,12657,2
...,...,...
53070,53070,0
53071,53071,0
53072,53072,0
53073,53073,0


<p>Посчитаем, в скольких матчах игрок долго бездействовал:</p>

In [16]:
afk = players[["match_id", "account_id", "leaver_status"]]
afk["leaver_status"] = players["leaver_status"].mask(players["leaver_status"].ne(4))
afk = afk.groupby("account_id", as_index=False)["leaver_status"].count()
afk.rename(columns = {'leaver_status':'afk'}, inplace = True)
afk.sort_values(by='afk', ascending=False)

,account_id,afk
0,0,299
11522,11522,2
30092,30092,1
94450,94450,1
101265,101265,1
...,...,...
52845,52845,0
52846,52846,0
52847,52847,0
52848,52848,0


<p>Объединим полученные данные по каждому игроку:</p>

In [17]:
joined = fouls.join(disconnected.set_index('account_id'), on='account_id')
joined = joined.join(long_disconnected.set_index('account_id'), on='account_id')
joined = joined.join(left.set_index('account_id'), on='account_id')
joined = joined.join(afk.set_index('account_id'), on='account_id')
joined.sort_values(by = ['profanity', 'disconnected', 'long_disconnected', 'left', 'afk'], ascending = [False, False, False, False, False], na_position = 'first')

,account_id,profanity,disconnected,long_disconnected,left,afk
0,0.0,36622.0,2353,1282,634,299
2337,2962.0,162.0,3,0,0,0
7010,9149.0,83.0,0,1,0,0
15124,20197.0,79.0,0,0,0,0
13043,17315.0,59.0,1,0,0,0
...,...,...,...,...,...,...
108030,158344.0,0.0,0,0,0,0
108031,158347.0,0.0,0,0,0,0
108033,158349.0,0.0,0,0,0,0
108036,158355.0,0.0,0,0,0,0


<h2>Вычисление токсичности</h2>

<p>Вычислим токсичность игроков по описанной в отчете формуле:</p>

In [18]:
results = joined.apply(lambda x: round(20*((x.profanity / 4)**2) + 50*((x.disconnected / 5)**5) + 500*((x.long_disconnected + x.left + x.afk)**1.5)), axis=1)
results = results.rename("toxicity")

In [19]:
joined = joined.join(results)
joined = joined[joined["account_id"] != 0]
joined.sort_values(by="toxicity", ascending=False)

,account_id,profanity,disconnected,long_disconnected,left,afk,toxicity
2337,2962.0,162.0,3,0,0,0,32809
7010,9149.0,83.0,0,1,0,0,9111
15124,20197.0,79.0,0,0,0,0,7801
13043,17315.0,59.0,1,0,0,0,4351
5095,6551.0,52.0,2,0,0,0,3381
...,...,...,...,...,...,...,...
44622,60583.0,0.0,0,0,0,0,0
44621,60582.0,0.0,0,0,0,0,0
44620,60581.0,0.0,0,0,0,0,0
44619,60580.0,0.0,0,0,0,0,0


In [20]:
joined.to_csv("/data/notebook_files/toxicity.csv")